In [2]:
%pip install pandas
%pip install -U matplotlib
%pip install seaborn
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 26.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.4
    Uninstalling matplotlib-3.8.4:
      Successfully uninstalled matplotlib-3.8.4
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.6 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 45.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 7.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 27.1 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os

Matplotlib is building the font cache; this may take a moment.


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
#with open('enriched_playlist.json', 'r', encoding='utf-8') as f:
    #data = f.read()

In [ ]:
#data = data.replace('\r\n', '\n')

In [ ]:
#data = data.replace('}\n        {', '},\n        {')

In [ ]:
#try:
   # json_data = json.loads(data)
    #print("JSON data loaded successfully")
#except json.JSONDecodeError as e:
    #print("JSON Decode Error:", e)

In [3]:
csv_file_path = 'enriched_playlist_v2.csv'  # Replace with your uploaded CSV file path

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)


In [4]:
print(df.head())

   playlist_id  num_followers                track_id  \
0          151              1  37eC6DX0aYrFGw8cMQaBG5   
1          151              1  142vj1HAI7VBAsn7OPmSHc   
2          151              1  5TAf4lnZCZTLlZHNZMLFLi   
3          151              1  4bJbSERtCqK9D6DRUPMmMh   
4          151              1  62YwtIlJIhRvFG7ChOCZSC   

                      track_name    artist_name  danceability  energy  key  \
0  Don't You Worry 'Bout A Thing  Stevie Wonder         0.714   0.726   11   
1                  Crazy For You          Adele         0.705   0.144    7   
2                          I Try      Macy Gray         0.593   0.642    2   
3          A Love That Will Last  Renee Olstead         0.666   0.303   11   
4                        If I Go      Ella Eyre         0.538   0.788    7   

   loudness  mode  speechiness  acousticness  instrumentalness  liveness  \
0    -9.218     1       0.0408        0.0316          0.000000    0.0666   
1   -12.234     1       0.0504      

In [4]:
print(df.describe())

        playlist_id  num_followers  danceability        energy           key  \
count  1.020120e+06   1.020120e+06  1.020120e+06  1.020120e+06  1.020120e+06   
mean   4.949506e+05   8.220923e+00  5.555361e-01  4.800764e-01  5.041395e+00   
std    2.894423e+05   5.464335e+02  1.590818e-01  2.454108e-01  3.545693e+00   
min    1.510000e+02   1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    2.457880e+05   1.000000e+00  4.510000e-01  2.840000e-01  2.000000e+00   
50%    4.867090e+05   1.000000e+00  5.650000e-01  4.750000e-01  5.000000e+00   
75%    7.477030e+05   2.000000e+00  6.660000e-01  6.770000e-01  8.000000e+00   
max    9.999050e+05   5.351900e+04  9.910000e-01  1.000000e+00  1.100000e+01   

           loudness          mode   speechiness  acousticness  \
count  1.020120e+06  1.020120e+06  1.020120e+06  1.020120e+06   
mean  -1.053208e+01  6.767096e-01  6.437435e-02  4.856984e-01   
std    4.807015e+00  4.677328e-01  6.932985e-02  3.432283e-01   
min   -6.000000e+01

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020120 entries, 0 to 1020119
Data columns (total 17 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   playlist_id       1020120 non-null  int64  
 1   num_followers     1020120 non-null  int64  
 2   track_id          1020120 non-null  object 
 3   track_name        1020120 non-null  object 
 4   artist_name       1020120 non-null  object 
 5   danceability      1020120 non-null  float64
 6   energy            1020120 non-null  float64
 7   key               1020120 non-null  int64  
 8   loudness          1020120 non-null  float64
 9   mode              1020120 non-null  int64  
 10  speechiness       1020120 non-null  float64
 11  acousticness      1020120 non-null  float64
 12  instrumentalness  1020120 non-null  float64
 13  liveness          1020120 non-null  float64
 14  valence           1020120 non-null  float64
 15  tempo             1020120 non-null  float64
 16  

In [ ]:
# Group by playlist and calculate descriptive statistics
playlist_stats = df.groupby('playlist_id').agg({
    'danceability': ['mean', 'std'],
    'energy': ['mean', 'std'],
    'valence': ['mean', 'std'],
    'tempo': ['mean', 'std'],
    'loudness': ['mean', 'std'],
    'speechiness': ['mean', 'std'],
    'acousticness': ['mean', 'std'],
    'instrumentalness': ['mean', 'std'],
    'liveness': ['mean', 'std']
}).reset_index()

print(playlist_stats)

In [ ]:
#Because of the data size, i will plot only a sample
playlist_id_sample = df_tracks['playlist_id'].sample(n=1, random_state=42).iloc[0]
df_sampled = df_tracks[df_tracks['playlist_id'] == playlist_id_sample]

# Define the features to plot
features = ['danceability', 'energy', 'valence', 'tempo', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness']

# Plot each feature individually using the sampled data
for feature in features:
    try:
        plt.figure(figsize=(10, 5))
        sns.histplot(data=df_sampled, x=feature, kde=True, binwidth=0.1)
        plt.title(f'Distribution of {feature} for Playlist {playlist_id_sample}')
        plt.tight_layout()
        plt.show()
    except ValueError as e:
        print(f"Could not plot {feature} due to: {e}")

In [5]:
# Separate features for normalization and standardization
features_to_normalize = ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']
features_to_standardize = ['loudness', 'tempo']

# Prepare features and labels
X = df[['danceability', 'energy', 'loudness', 'speechiness',
          'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
y = df['playlist_id']

# Apply MinMaxScaler for normalization
scaler_norm = MinMaxScaler()
X[features_to_normalize] = scaler_norm.fit_transform(X[features_to_normalize])

# Apply StandardScaler for standardization
scaler_std = StandardScaler()
X[features_to_standardize] = scaler_std.fit_transform(X[features_to_standardize])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Output the shapes of the training and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


/tmp/ipykernel_847/7247273.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[features_to_normalize] = scaler_norm.fit_transform(X[features_to_normalize])
/tmp/ipykernel_847/7247273.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[features_to_standardize] = scaler_std.fit_transform(X[features_to_standardize])


(816096, 9) (204024, 9) (816096,) (204024,)


In [6]:
# Initialize the model
clf = RandomForestClassifier(n_estimators=10, random_state=42)

# Train the model
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=42)

In [ ]:
test_accuracy = clf.score(X_test, y_test)

In [ ]:
print(f"Test set accuracy: {test_accuracy:.2f}")